In [18]:
# Get data out of the file
def get_edges(line):
    edges = []
    line = line.translate(None,"[]")
    for token in line.split(")"):
        if token:
            values = token.split("(")[1].split(",")
            edges.append((int(values[0]),int(values[1])))
    return edges 
# make pairs (g,i)
data = []
from collections import defaultdict,deque
f = open('input.txt','r')
for line in f:
    if line != '\n':
        strings = line.split("]")
        data.append([get_edges(strings[0]),get_edges(strings[1])])
f.close()
class Graph:
    def __init__(self,edges,requests):
        self.edges = edges
        self.inputs = requests 
        self.adj_list = self.adjacency_list()
    def adjacency_list(self):
        adj_list = defaultdict(set)
        for start,end in self.edges:
            adj_list[start].add(end)
            adj_list[end].add(start)
        return adj_list
    def all_unvisited(self):
        visited = defaultdict(int)
        for v in self.adj_list:
            visited[v]=0 # we know that no vertices start from 0
        return visited
    def remove_edge(self,edge):
        s,b = edge[0],edge[1]
        # remove this guy from edges 
        tup = (s,b) if s<b else (b,s)
        if tup in self.edges:
            self.edges.remove(tup)
            self.adj_list[s].discard(b)
            self.adj_list[b].discard(s)
        # adjust the adj list
        

In [2]:
def shortest_path(graph,start,end):
        def edges_from_path(path):
            if len(path)<=1:
                return path
            edges = []
            for i in range(len(path)-1):
                tup = (path[i],path[i+1]) if path[i] < path[i+1] else (path[i+1],path[i])
                edges.append(tup)
            return edges 
        
        pending =  deque([start])
        visited = graph.all_unvisited()
        path = []
        while pending:
            current = pending.popleft()
            if current == end:
                # track back all the neighbors 
                while current != start:
                    path.append(current)
                    current = visited[current]
                path.append(current)
                path.reverse()
                return edges_from_path(path)
            
            for neighbor in graph.adj_list[current]:
                if visited[neighbor] == 0:
                    visited[neighbor] = current
                    pending.append(neighbor)
        return edges_from_path(path)

In [147]:
def graph_coloring(adj_list):
    # assume that adj_list is int:set, the vertices are marked from 1 to len(adj_list)
    total_colors = 0
    all_vertices = set(adj_list.keys())
    colors = defaultdict(int)
    colors_adj_list = defaultdict(set)
    # since this is defaultdict, it would return 0 and empty set if the emements are not present in the dict for the keys!
    # Algorithm in a nutshell:
    # http://www.geeksforgeeks.org/graph-coloring-set-2-greedy-algorithm/
    if not adj_list:
        #total_colors = 1
        pass
    else: 
        queue = deque([min(adj_list.keys())])
        #queue = deque(max([(k,len(v)) for k,v in adj_list.iteritems()],key = lambda (x,y):y)[0])
        while queue:
            c = queue.popleft()
            if not colors[c]:
                # this guy is not colored
                pick = sorted(list(all_vertices - colors_adj_list[c]))[0]
                colors[c] = pick
                total_colors = pick if total_colors < pick else total_colors
                for neighbor in adj_list[c]:
                    if not colors[neighbor]:
                        # if this guy is not colored yet
                        colors_adj_list[neighbor].add(pick)
                        queue.append(neighbor)
            if not queue:
                for vertex in adj_list.keys():
                    if colors[vertex] == 0:
                        queue.append(vertex)
                        break
        for vertex in colors.keys():
            colors[vertex]=1/float(colors[vertex])
    colors["total_colors"] = total_colors
    return colors 


In [122]:
def remove_paths(g,paths):
    for r in paths.keys():
        for edge in paths[r]:
            g.remove_edge(edge)
            
def remove_list_of_edges(g,edges):
    for edge in edges:
        g.remove_edge(edge)
            
def make_shortest_paths(g):
            shortestPaths=defaultdict(set)
            for s,t in g.inputs:
                shortestPaths[(s,t)] = set(shortest_path(g,s,t))
            return shortestPaths
        
def make_graph_of_demands(demand_paths):
        translate = sorted(demand_paths.keys()) # use indices as back and forth translation 
        adj_list = defaultdict(set)
        for i in range(len(translate)):
            for j in range(i+1,len(translate)):
                if i != j and demand_paths[translate[i]].intersection(demand_paths[translate[j]]):
                    adj_list[i+1].add(j+1)
                    adj_list[j+1].add(i+1)
        return adj_list

def number_of_different_wavelengths(adj_list):
    #adj_list = make_graph_of_demands(g)
    #edges = edges_from_adj_list(adj_list)
    color_map =graph_coloring(adj_list)
    #draw_graph(edges,color_map)
    return color_map['total_colors']

In [8]:
########################################################################
## USAGE for unit testing only###
def edges_from_adj_list(adj_list):
    edges = []
    for vertex in adj_list:
        for neighbor in adj_list[vertex]:
            if vertex < neighbor:
                edges.append((vertex,neighbor))
    return edges
import networkx as nx
import matplotlib.pyplot as plt
def draw_graph(edges_list, values_map = None):
    G=nx.Graph()
    G.add_edges_from(edges_list)
    if not values_map:
        nx.draw(G,with_labels = True,node_color='Aqua')
    else:
        values = [values_map[node] for node in G.nodes()]
        nx.draw(G, cmap=plt.get_cmap('cool'), node_color=values,with_labels = True)
    plt.show()

In [153]:
def modify_old_paths(oldpaths,new_graph):
    requests = sorted(oldpaths.keys())
    for i,r in enumerate(requests):
        flag = 0 
        for j in range(i):
            if oldpaths[requests[j]].intersection(oldpaths[r]) and flag == 0:
                #find a new path for r in the new graph if possible 
                possible_path = set(shortest_path(new_graph,r[0],r[1]))
                if possible_path:
                    # yes there is a path for the request r in the new_graph, set this as a pat andremove edges from new graph
                    #oldpaths[r]= possible_path
                    oldpaths.pop(r,None)
                    flag = 1
                    remove_list_of_edges(new_graph,possible_path)
                    #print r,possible_path

In [8]:
answers = []
for line in data:
    #print line[0],"\n",line[1]
    #line[0] is the graph, line[1] is the requests 
    g = Graph(line[0],line[1])
    r = old_and_new_search(g)
    if r[0] > r[1]:
        print line
        break
    answers.append(r)
    #result = str()+";"+str(len(g.edges))+";"+str(len(g.inputs))+"\n"
    #f.write(result)
    #print result

[[(14, 17), (7, 12), (1, 17), (17, 20), (1, 6), (1, 11), (6, 7), (12, 17), (13, 20), (9, 14), (4, 5), (10, 13), (16, 19), (2, 17), (17, 18), (3, 18), (10, 14), (9, 19), (7, 8), (2, 18), (8, 9), (6, 14), (3, 6), (1, 10), (4, 11), (3, 5), (9, 13), (4, 6), (5, 7), (5, 20), (16, 20), (3, 15), (4, 8), (5, 13), (8, 19), (9, 18), (7, 11), (11, 19), (16, 17), (2, 19), (1, 14), (8, 10), (4, 13), (6, 15), (12, 14), (13, 15), (10, 16), (13, 18), (3, 4), (9, 12), (5, 9), (7, 16), (8, 14), (2, 9), (5, 12), (10, 12), (9, 17), (11, 18), (16, 18), (17, 19), (5, 15), (3, 19), (8, 17), (6, 12), (6, 19), (12, 15), (11, 17), (3, 8)], [(1, 2), (1, 4), (1, 5), (1, 9), (1, 12), (1, 16), (2, 6), (2, 8), (2, 12), (3, 9), (3, 12), (3, 17), (4, 9), (4, 12), (4, 16), (5, 8), (5, 10), (5, 14), (5, 18), (6, 8), (6, 9), (6, 16), (6, 17), (7, 15), (7, 19), (8, 12), (8, 16), (9, 15), (10, 18), (11, 12), (12, 13), (12, 18), (12, 20), (13, 17), (14, 16), (14, 18), (15, 17), (15, 18), (15, 20)]]


In [154]:
line = [[(14, 17), (7, 12), (1, 17), (17, 20), (1, 6), (1, 11), (6, 7), (12, 17), (13, 20), (9, 14), (4, 5), (10, 13), (16, 19), (2, 17), (17, 18), (3, 18), (10, 14), (9, 19), (7, 8), (2, 18), (8, 9), (6, 14), (3, 6), (1, 10), (4, 11), (3, 5), (9, 13), (4, 6), (5, 7), (5, 20), (16, 20), (3, 15), (4, 8), (5, 13), (8, 19), (9, 18), (7, 11), (11, 19), (16, 17), (2, 19), (1, 14), (8, 10), (4, 13), (6, 15), (12, 14), (13, 15), (10, 16), (13, 18), (3, 4), (9, 12), (5, 9), (7, 16), (8, 14), (2, 9), (5, 12), (10, 12), (9, 17), (11, 18), (16, 18), (17, 19), (5, 15), (3, 19), (8, 17), (6, 12), (6, 19), (12, 15), (11, 17), (3, 8)], [(1, 2), (1, 4), (1, 5), (1, 9), (1, 12), (1, 16), (2, 6), (2, 8), (2, 12), (3, 9), (3, 12), (3, 17), (4, 9), (4, 12), (4, 16), (5, 8), (5, 10), (5, 14), (5, 18), (6, 8), (6, 9), (6, 16), (6, 17), (7, 15), (7, 19), (8, 12), (8, 16), (9, 15), (10, 18), (11, 12), (12, 13), (12, 18), (12, 20), (13, 17), (14, 16), (14, 18), (15, 17), (15, 18), (15, 20)]]

In [155]:
g = Graph(line[0],line[1])

In [156]:
initial_paths  = make_shortest_paths(g)
initial_wavelengths = number_of_different_wavelengths(make_graph_of_demands(initial_paths))
initial_wavelengths

7

In [130]:
#sorted(g.edges)

In [131]:
#import copy
#tmp = copy.deepcopy(initial_paths)
#tmp

In [157]:
remove_paths(g,initial_paths)
#g.adj_list
#sorted(g.edges)

In [158]:
modify_old_paths(initial_paths,g)
#g.adj_list

In [159]:
initial_paths

defaultdict(set,
            {(1, 2): {(1, 17), (2, 17)},
             (1, 4): {(1, 11), (4, 11)},
             (1, 5): {(1, 17), (5, 9), (9, 17)},
             (1, 9): set(),
             (1, 12): set(),
             (1, 16): {(1, 17), (16, 17)},
             (2, 6): {(2, 19), (6, 19)},
             (2, 8): set(),
             (2, 12): {(2, 17), (12, 17)},
             (3, 9): {(3, 5), (5, 9)},
             (3, 12): {(3, 5), (5, 12)},
             (3, 17): {(3, 8), (8, 17)},
             (4, 9): {(4, 5), (5, 9)},
             (4, 12): {(4, 5), (5, 12)},
             (4, 16): {(3, 4), (3, 18), (16, 18)},
             (5, 8): {(3, 5), (3, 8)},
             (5, 10): {(5, 12), (10, 12)},
             (5, 14): {(5, 9), (9, 14)},
             (5, 18): {(3, 5), (3, 18)},
             (6, 8): set(),
             (6, 9): {(6, 12), (9, 12)},
             (6, 16): {(6, 7), (7, 16)},
             (6, 17): set(),
             (7, 15): {(5, 7), (5, 15)},
             (7, 19): {(6, 7), (6, 19)},
   

In [160]:
wavelengths = number_of_different_wavelengths(make_graph_of_demands(initial_paths))
wavelengths

9

In [41]:
initial_paths

defaultdict(set,
            {(1, 2): {(1, 17), (2, 17)},
             (1, 4): {(1, 11), (4, 11)},
             (1, 5): {(1, 17), (5, 9), (9, 17)},
             (1, 9): {(1, 17), (9, 17)},
             (1, 12): {(1, 17), (12, 17)},
             (1, 16): {(1, 17), (16, 17)},
             (2, 6): {(2, 19), (6, 19)},
             (2, 8): {(2, 17), (8, 17)},
             (2, 12): {(2, 17), (12, 17)},
             (3, 9): {(3, 5), (5, 9)},
             (3, 12): {(3, 5), (5, 12)},
             (3, 17): {(3, 19), (17, 19)},
             (4, 9): {(4, 5), (5, 9)},
             (4, 12): {(4, 5), (5, 12)},
             (4, 16): {(3, 4), (3, 18), (16, 18)},
             (5, 8): {(3, 5), (3, 8)},
             (5, 10): {(5, 12), (10, 12)},
             (5, 14): {(5, 9), (9, 14)},
             (5, 18): {(3, 5), (3, 18)},
             (6, 8): {(3, 6), (3, 8)},
             (6, 9): {(4, 6), (4, 8), (8, 9)},
             (6, 16): {(6, 7), (7, 16)},
             (6, 17): {(1, 6), (1, 17)},
             (